In [ ]:
import numpy as np, random, operator, pandas as pd

class Direction:
    def __init__(self, direction):
        self.direction = direction

    def __repr__(self):
        return str(self.direction)
    
class Fitness:
    def __init__(self, path):
        self.path = path
        self.distance = 0
        self.fitness = 0
        
        
    def calDistance(self):
        
        square.centerx, square.centery = WIDTH // 2, HEIGHT // 2
        
        for i in range(0, len(self.path)):

            #print(self.path[i]);
            
            gene = self.path[i]

            #print('gene', gene)
            collided = False

            screen.fill(BLACK)  

            dt = clock.tick(60)
            
            #1 cima
            #2 direita
            #3 baixo
            #4 esquerda
            
            #print('deslocamento', velocity * dt)
            #print('gene', gene)

            if (gene == 1):
                pos_y = -(velocity * dt)
                square.move_ip(0, pos_y)

            if (gene == 2):
                pos_x = velocity * dt
                square.move_ip(pos_x, 0)

            if (gene == 3):
                pos_y = (velocity * dt)
                square.move_ip(0, pos_y)

            if (gene == 4):
                pos_x = -(velocity * dt)
                square.move_ip(pos_x, 0)

            for wall in walls:
                if square.colliderect(wall):
                    print('bateu')
                    collided = True

            if (collided == True):
                break;
            
            for wall in walls:
                pygame.draw.rect(screen, WHITE, wall)

            pygame.draw.rect(screen, RED, square)
            
            
            init_x, init_y = WIDTH // 2, HEIGHT // 2
            current_x, current_y = square.centerx, square.centery
                
            xDis = abs(current_x - init_x)
            #calcula a distancia dos eixos y
            yDis = abs(current_y - init_y)
            #eleva as diferenças ao quadrado, soma e calcula a raiz
            self.distance = np.sqrt((xDis ** 2) + (yDis ** 2))
            
            str_mov = 'distance = ' + str(round(self.distance, 2))
            text_mov = font.render(str_mov, True, WHITE)
            text_rect = text_mov.get_rect()
            text_rect.center = (WIDTH // 2, HEIGHT - 60)

            screen.blit(text_mov, text_rect)
            
            pygame.display.flip()

        return self.distance
        
    def getPathFitness(self):
        if (self.fitness == 0):
            distance = int(self.calDistance())
            if (distance > 0):
                self.fitness = 1 / distance
            self.fitness = 1
        return self.fitness

    def getDistance(self):
        return self.distance



import pygame
import sys

BLACK = pygame.Color(0, 0, 0)
WHITE = pygame.Color(255, 255, 255)
RED = pygame.Color(255, 0, 0)
WIDTH, HEIGHT = 600, 300

SQUARE_WIDTH, SQUARE_HEIGHT = 5, 5
WALL_WIDTH, WALL_HEIGHT = 20, 20

velocity = 1

pygame.init()

screen = pygame.display.set_mode((WIDTH, HEIGHT))

clock = pygame.time.Clock()

square = pygame.Rect(WIDTH // 2, HEIGHT // 2, SQUARE_WIDTH, SQUARE_HEIGHT)


levels = [
    "WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW",
    "W                            W",
    "W      WWWWWWWWWW            W",
    "W      W        W            W",
    "W      W     W  W            W",
    "W      W     W  W            W",
    "W      W     W  W            W",
    "W      W     W  W            W",
    "W      W     W  W            W",
    "W      W   WWWWWW            W",
    "W      W   W                 W",
    "W      W   W                 W",
    "W      W                     W",
    "W                            W",
    "WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW"
]


left = 0
top = 0

walls = []

for row in levels:
    for col in row:
        if col == 'W':
            wall = pygame.Rect(left, top, WALL_WIDTH, WALL_HEIGHT)
            walls.append(wall)
        left += WALL_WIDTH
    top += WALL_HEIGHT
    if left >= WIDTH:
        left = 0


font = pygame.font.Font('freesansbold.ttf', 12)
        
class GeneticAlgorithm:
    def __init__(self, population, populationSize, eliteSize, mutationRate, generations):
        self.population = population
        self.populationSize = populationSize
        self.eliteSize = eliteSize
        self.mutationRate = mutationRate
        self.generations = generations
        self.currentPopulation = []
        self.bestFitnessMovimentations = 1
        self.bestDistance = 0

    def start(self):
        self.currentPopulation = self.initialPopulation(self.populationSize, self.population)

        for i in range(0, self.generations):
            print('generation: ', i)
            self.currentPopulation = self.nextGeneration(self.currentPopulation, self.eliteSize, self.mutationRate)

        bestIndividualIndex = self.rankIndividuals(self.currentPopulation)[0][0]
        bestIndividual = self.currentPopulation[bestIndividualIndex]
        print(bestIndividual)
        
        
    def createPath(self, listDirections):
        return random.sample(listDirections, len(listDirections))


    def initialPopulation(self, populationSize, listDirections):
        population = []

        for i in range(0, populationSize):
            population.append(self.createPath(listDirections))
        return population


    def rankIndividuals(self, population):
        fitnessResults = {}
        
        for i in range(0, len(population)):
            fitnessResults[i] = Fitness(population[i]).getPathFitness()
            
        return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)


    def selection(self, populationRanked, eliteSize):
        #print('populationRanked', populationRanked)

        selectionResults = []
        df = pd.DataFrame(np.array(populationRanked), columns=["Index", "Fitness"])
        df['cum_sum'] = df.Fitness.cumsum()
        df['cum_perc'] = 100 * df.cum_sum / df.Fitness.sum()

        for i in range(0, eliteSize):
            selectionResults.append(populationRanked[i][0])

        for i in range(0, len(populationRanked) - eliteSize):
            pick = 100 * random.random()

            for i in range(0, len(populationRanked)):
                if pick <= df.iat[i, 3]:
                    selectionResults.append(populationRanked[i][0])
                    break
        return selectionResults

    def matingPool(self, population, selectionResults):
        matingpool = []

        for i in range(0, len(selectionResults)):
            index = selectionResults[i]
            matingpool.append(population[index])
        return matingpool

    def breed(self, parent1, parent2):
        child = []
        childP1 = []
        childP2 = []

        geneA = int(random.random() * len(parent1))
        geneB = int(random.random() * len(parent1))

        startGene = min(geneA, geneB)
        endGene = max(geneA, geneB)

        for i in range(startGene, endGene):
            childP1.append(parent1[i])

        childP2 = [item for item in parent2 if item not in childP1]

        child = childP1 + childP2

        return child

    def breedPopulation(self, matingpool, eliteSize):
        children = []
        length = len(matingpool) - eliteSize
        pool = random.sample(matingpool, len(matingpool))

        for i in range(0, eliteSize):
            children.append(matingpool[i])

        for i in range(0, length):
            parent1 = pool[i]
            parent2 = pool[len(matingpool)-i-1]
            child = self.breed(parent1, parent2)
            children.append(child)

        return children


    def mutate(self, individual, mutationRate):
        for geneIndex in range(len(individual)):
            if (random.random() < mutationRate):

                newDirection = random.randint(1, 4)
                individual[geneIndex] = newDirection

        return individual

    def mutatePopulation(self, population, mutationRate):
        mutatedPopulation = []

        for individual in range(0, len(population)):

            mutatedIndividual = self.mutate(population[individual], mutationRate)
            mutatedPopulation.append(mutatedIndividual)

        return mutatedPopulation

    def nextGeneration(self, currentGeneration, eliteSize, mutationRate):
        populationRanked = self.rankIndividuals(currentGeneration)
        selectionResults = self.selection(populationRanked, eliteSize)
        matingpool = self.matingPool(currentGeneration, selectionResults)
        children = self.breedPopulation(matingpool, eliteSize)
        return self.mutatePopulation(children, mutationRate) 


while True:

    events = pygame.event.get()
    
    for event in events:
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_SPACE:
                listDirections = []

                for i in range(0, 6):
                    listDirections.append(Direction(direction=random.random()))
                    
                ga = GeneticAlgorithm(population=listDirections, populationSize=5, eliteSize=2, mutationRate=0.1, generations=50)
                ga.start()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
generation:  0
generation:  1
generation:  2
bateu
generation:  3
generation:  4
generation:  5
generation:  6
generation:  7
bateu
generation:  8
bateu
bateu
generation:  9
bateu
bateu
generation:  10
bateu
bateu
bateu
bateu
generation:  11
bateu
generation:  12
bateu
bateu
bateu
bateu
bateu
generation:  13
bateu
bateu
bateu
bateu
bateu
bateu
bateu
generation:  14
bateu
bateu
bateu
generation:  15
bateu
bateu
bateu
bateu
bateu
bateu
generation:  16
bateu
